# Système non-linéaires

## Méthode de Newton

Optimisation:
$$
x_{k+1} = x_k-\nabla^2 f(x_k)^{-1} \nabla f(x_k)
$$
ou
$$
\nabla^2 f(x_k) x_{k+1} = \nabla^2 f(x_k) x_k- \nabla f(x_k)
$$
C'est équivalent à chercher $x_k$ tel que $\nabla f(x_k) = 0$.

La méthode de Newton peut aussi être appliquée à des systèmes nonlinéaires plus généraux.

In [ ]:
using LinearAlgebra

In [ ]:
function NLSNewton(g::Function, h:: Function,
        xstart::Vector, δ::Float64 = 1e-4, nmax::Int64 = 1000)

    k = 1
    x = xstart
    n = length(x)
    δ2 = δ*δ
    H = zeros(n,n)
    dfx = ones(n)
    
    g(x, dfx)

    while (dot(dfx,dfx) > δ2 && k <= nmax)
        k += 1
        g(x,dfx)
        h(x,H)
        # println(x, dfx, H)
        # Hs = dfx, x_{k+1} = x_k - s
        x -= H\dfx  # x = x - s
    end
    
    return x
end

Considérons par exemple le système non-linéaire
\begin{align*}
    4x+2y+2xz-10 &= 0 \\
    2x+y+yz-6 &= 0 \\
    x^2 + y^2 - 5 &= 0
\end{align*}
Nous implémentons d'abord la multi-fonction correspondante.

In [ ]:
function g(x::Vector, d::Vector)
    d[1] = 4*x[1]+2*x[2]+2*x[1]*x[3]-10
    d[2] = 2*x[1]+x[2]+x[2]*x[3]-6
    d[3] = x[1]*x[1]+x[2]*x[2]-5    
end

Nous pouvons facilement vérifier que (1,2,1) satisfait ce système.

In [ ]:
d = zeros(3)
x = [ 1; 2; 1]
h = g(x, d)

La matrice jacobienne est
$$
\begin{pmatrix}
    4+2z & 2 & 2x \\
    2 & 1+2z & y \\
    2x & 2y & 0
\end{pmatrix}
$$

In [ ]:
function J(x::Vector, H::Matrix)
    H[1,1] = 4+2*x[3]
    H[2,2] = 2+2*x[3]
    H[3,3] = 0.0
    H[1,2] = H[2,1] = 2
    H[1,3] = H[3,1] = 2*x[1]
    H[2,3] = x[2]
    H[3,2] = 2*x[2]
end

In [ ]:
NLSNewton(g, J, x)

In [ ]:
x0 = [1,1,1.0]

In [ ]:
NLSNewton(g, J, x0)

La méthode de Newton pour résoudre un système non-linéaire souffre cependant des mêmes problèmes que dans le contexte d'optimisation. Sa convergence n'est assurée que si le point de départ est dans un voisinage de la solution. La récurrence peut même être ne pas être bien définie.

In [ ]:
NLSNewton(g, J, [0, 0 ,0.0])

## Exemple 2

Nous cherchons à résoudre le problème
\begin{align*}
x^3 + y &= 1 \\
y^3 − x &= −1.
\end{align*}

La multi-fonction du sytème peut être définie par

In [ ]:
function g2(x::Vector, d::Vector)
    d[1] = x[1]^3+x[2]-1
    d[2] = x[2]^3-x[1]+1
    return d
end

In [ ]:
d = [1.0, 1.0]
x = [1.0, 0]

In [ ]:
g2(x, d)
d

In [ ]:
function J2(x::Vector, H::Matrix)
    H[1,1] = 3*x[1]^2
    H[1,2] = 1
    H[2,1] = -1
    H[2,2] = 3*x[2]^2
    return H
end

In [ ]:
H = [0 0; 0 0]
J2(x,H)

In [ ]:
x = [1.0, 1.0]
NLSNewton(g2, J2, x)

Réalisaons la première itération à la main. Nous devons résoudre le système
$$
\begin{pmatrix}
3*1^2 & 1 \\ -1 & 3*1^2
\end{pmatrix}
\begin{pmatrix}
d_1 \\ d_2
\end{pmatrix}
=
\begin{pmatrix}
1^1+1-1 \\ 1^3-1+1
\end{pmatrix}
$$
ou
$$
\begin{pmatrix}
3 & 1 \\ -1 & 3
\end{pmatrix}
\begin{pmatrix}
d_1 \\ d_2
\end{pmatrix}
=
\begin{pmatrix}
1 \\ 1
\end{pmatrix}
$$
De manière équivalente, nous devons résoudre
\begin{align*}
3d_1 + d_2 &= 1 \\
-d_1 + 3d_2 &= 1 \\
\end{align*}
Nous en déduisons que
$$
10 d_2 = 4
$$
ou
$$
d_2 = 0.4
$$
et dès lors
$$
d_1 = 0.2
$$
Nous pouvons vérifier ce résultat en résolvant numériquement le système linéaire correspondant

In [ ]:
A = [3 1 ; -1 3]
b = [1 ; 1]
A\b

Dès lors, l'itération de Newton donne
$$
\begin{pmatrix}
x_{k+1} \\ y_{k+1}
\end{pmatrix}
=
\begin{pmatrix}
1 \\ 1
\end{pmatrix}
-
\begin{pmatrix}
0.2 \\ 0.4
\end{pmatrix}
=
\begin{pmatrix}
0.8 \\ 0.6
\end{pmatrix}
$$

## Exemple 3

\begin{align*}
a+b+c &= 6 \\
a^2+b^2+c^2 &= 14 \\
a^3+b^3+c^3 &= 36
\end{align*}

In [ ]:
function g3(x::Vector, d::Vector)
    for i = 1:3
        d[i] = x[1]^i+x[2]^i+x[3]^i
    end
    d[1] -= 6
    d[2] -= 14
    d[3] -= 36
    return d
end

In [ ]:
function J3(x::Vector, H::Matrix)
    for i = 1:3
        for j = 1:3
            H[i,j] = i*x[j]^(i-1)
        end
    end
    return H
end

In [ ]:
x = [1.0, 1.0, 1.0]
NLSNewton(g3, J3, x)

In [ ]:
x = [2.0, 1.0, 0.0]
NLSNewton(g3, J3, x)

## Application au conditions KKT

Considérons le problème contraint linéairement
\begin{align*}
\min\ & f(x) \\
\mbox{t.q. } & Ax = b
\end{align*}
où $A \in \mathbb{R}^{m \times n}$.

La fonction lagrangienne de ce problème est
$$
L(x,\mu) = f(x) + \sum_{i = 1}^m \mu_i(a_i^Tx-b_i)
$$
où $a_i$ désigne la $i^e$ ligne de la matrice et $b_i$ le $i^e$ élément de $b$.

Les conditions KKT de ce problème sont
\begin{align*}
\nabla f(x) + A^T\mu &= 0 \\
Ax - b &= 0
\end{align*}
Il s'agit d'un système non-linéaire qui peut être résolu en utilisant la méthode de Newton.

Considérons par exemple le problème
$$
\min f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [ ]:
f = x -> -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4
function ∇f(x:: Vector, g:: Vector)
    g[1] = -20*x[1]+4*x[2]*cos(x[1]*x[2])-2+4*x[1]^3
    g[2] = 20*x[2]+4*x[1]*cos(x[1]*x[2])
    return g
end
function Hess(x:: Vector, H:: Matrix)
    H[1,1] = -20-4*x[2]^2*sin(x[1]*x[2])+12*x[1]^2
    H[2,1] = H[1,2] = 4*cos(x[1]*x[2])-4*x[1]*x[2]*sin(x[1]*x[2])
    H[2,2] = 20-4*x[1]^2*sin(x[1]*x[2])
    return H
end

In [ ]:
using ForwardDiff

gr = x -> ForwardDiff.gradient(f, x);
He = x -> ForwardDiff.hessian(f, x)

function gr!(x::Vector, storage::Vector)
    s = gr(x)
    storage[1:length(s)] = s[1:length(s)]
end

function He!(x::Vector, storage::Matrix)
    s = He(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:n,1:m]
end

In [ ]:
f(x)

In [ ]:
x = [2.0; -3.0]

In [ ]:
grad = zeros(2)
∇f(x,grad)

In [ ]:
gr!(x,grad)

In [ ]:
hess = zeros(2,2)
Hess(x,hess)

In [ ]:
He!(x,hess)
hess

In [ ]:
sol = NLSNewton(∇f, Hess, x)

In [ ]:
sol = NLSNewton(gr!, He!, x)

In [ ]:
∇f(sol, grad)

In [ ]:
Hess(sol, hess)

Nous avons identifié un point selle!

Commençons avec un autre point de départ.

In [ ]:
sol = [-2.21022, 0.329748]
∇f(sol, grad)

In [ ]:
x = [-10; -10.0]
sol = NLSNewton(gr!, He!, x)

In [ ]:
sol = NLSNewton(∇f, Hess, x)

In [ ]:
sol = [2.30663, -0.332309]
∇f(sol, grad)

In [ ]:
x = [2.5, -0.3]
sol = NLSNewton(gr!, He!, x)

In [ ]:
∇f(sol, grad)

Considérons maintenant le programme contraint
\begin{align*}
\min\ & f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4 \\
\text{s.t. } & 0.1x+y=1
\end{align*}
La fonction lagrangienne est maintenant
$$
L(x,y,\mu) = -10x^2+10y^2+4\sin(xy)-2x+x^4 + \mu(0.1x+y-1)
$$
et les conditions KKT sont
\begin{align*}
-20x+4y\cos(xy)-2+4x^3 + 0.1\mu &= 0 \\
20y+4x\cos(xy) + \mu &= 0 \\
0.1x+y-1 &= 0
\end{align*}

Afin de résoudre ce système, développons la matrice jacobienne
$$
J(x,y,\mu) =
\begin{pmatrix}
-20 - 4y\sin(xy)+12x^2 & 4\cos(xy)-4xy\cos(xy) & 0.1 \\
4\cos(xy)-4xy\cos(xy) & 20 - 4x^2\sin(xy) & 1 \\
0.1 & 1 & 0
\end{pmatrix}
$$

Implémentons ces opérateurs.

In [ ]:
function L!(x, L)
    L[1] = -20*x[1]+4*x[2]*cos(x[1]*x[2])-2+4*x[1]^3+0.1*x[3]
    L[2] = 20*x[2]+4*x[1]*cos(x[1]*x[2])+x[3]
    L[3] = 0.1*x[1]+x[2]-1
    return L
end

function J!(x, J)
    J[1,1] = -20-4*x[2]^2*sin(x[1]*x[2])+12*x[1]^2
    J[2,1] = J[1,2] = 4*cos(x[1]*x[2])-4*x[1]*x[2]*sin(x[1]*x[2])
    J[2,2] = 20-4*x[1]^2*sin(x[1]*x[2])
    J[3,3] = 0
    J[1,3] = J[3,1] = 0.1
    J[2,3] = J[3,2] = 1
    return J
end

In [ ]:
x = [2.5, -0.3, 1.0]
sol = NLSNewton(L!, J!, x)

In [ ]:
x = [-2.5, -2.5, 1.0]
sol = NLSNewton(L!, J!, x)

Nous avons trouvé deux solutions au problème KKT.

Notons que dans cet exemple, nous pouvons aussi utiliser la contrainte linéaire pour éliminer une variable dans la fonction objectif, et avoir seulement à résoudre un programme à une dimension.

Deux difficultés demeurent.

Tout d'abord, nous devons "globaliser" la méthode, i.e. s'assurer qu'elle converge à partir de n'importe quel point de départ.

Ensuite, afin de traiter avec les contraintes d'inégalité, nous devons être capable de déterminer l'ensemble actif à la solution.